In [7]:
import pandas as pd
from scipy.spatial import distance
from scipy.stats import chi2
import numpy as np
import plotly.express as px

In [9]:
# results from pca dimensionality reduction 
df = pd.read_csv("../asteraceae_pca_astera_50_checkpoint-1300.csv")
df.head()

,Unnamed: 0,0,1,label
0,/projectnb/herbdl/data/kaggle-herbaria/herbari...,9.106315,0.102141,Asteraceae Ageratina jucunda
1,/projectnb/herbdl/data/kaggle-herbaria/herbari...,10.195288,-11.720784,Asteraceae Carphephorus paniculatus
2,/projectnb/herbdl/data/kaggle-herbaria/herbari...,-5.752719,-7.106326,Asteraceae Crepis acuminata
3,/projectnb/herbdl/data/kaggle-herbaria/herbari...,3.561659,14.624518,Asteraceae Bidens vulgata
4,/projectnb/herbdl/data/kaggle-herbaria/herbari...,-12.460205,-4.621024,Asteraceae Arnica sororia


In [10]:
# get list of labels - species 
species_names = df["label"].unique()

In [11]:
# for each label filter the df for the points 
result_dict = {}
for name in species_names:
    result_dict[name] = df[df["label"] == name]

In [12]:
# calculate the centroid for all points in the cluster 
# return the centroid for the cluster
def calculate_centroid(results_df: pd.DataFrame) -> tuple:
    x_s = results_df["0"]
    y_s = results_df["1"]
    group_size = len(x_s)
    center_x = sum(x_s)/group_size
    center_y = sum(y_s)/group_size 
    centroid = (center_x, center_y)
    return centroid

In [13]:
# calculate the distance to the centroid for each point in the cluster
# points_df: the DataFrame containing the points in columns '0' and '1' 
# centroid: the calculated centroid the cluster/group
# return the distances for the each point in the group as a list
def calculate_distances(points_df: pd.DataFrame, centroid: tuple) -> list:
    indexes = points_df.index 
    group_distances = []
    
    for idx in indexes:
        point = [points_df.loc[idx]["0"], points_df.loc[idx]["1"]]
        eucl_distance = distance.euclidean(point, centroid)
        group_distances.append(eucl_distance) 

    return group_distances

In [49]:
# write the distances to the original dataframe 
# calculate +/- 2 standard deviations 
# return the top threshold and bottom threshold
def calculate_thresholds(group_distances: list):
    top_thresh = np.mean(group_distances) + (np.std(group_distances)*2) 
    bot_thresh = np.mean(group_distances) - (np.std(group_distances)*2)
    return top_thresh, bot_thresh

In [15]:
# calculate outliers for each species 
centroids_dict = {}
for name in species_names:
    print(f"calculating outlier for group {name}")
    center = calculate_centroid(result_dict[name])
    distances = calculate_distances(result_dict[name], center)
    h_thresh, l_thresh = calculate_thresholds(distances) 

    # save the centroids
    centroids_dict[name] = center
    
    # get the outliers 
    outliers = [True if d > h_thresh or d < l_thresh else False for d in distances]
    
    # save the results to the dataframe 
    result_dict[name].loc[:, "eucl_distance"] = distances 
    result_dict[name].loc[:, "outlier"] = outliers

calculating outlier for group Asteraceae Ageratina jucunda
calculating outlier for group Asteraceae Carphephorus paniculatus
calculating outlier for group Asteraceae Crepis acuminata
calculating outlier for group Asteraceae Bidens vulgata
calculating outlier for group Asteraceae Arnica sororia
calculating outlier for group Asteraceae Brickellia longifolia
calculating outlier for group Asteraceae Liatris pilosa
calculating outlier for group Asteraceae Bidens beckii
calculating outlier for group Asteraceae Heterotheca pumila
calculating outlier for group Asteraceae Erigeron melanocephalus
calculating outlier for group Asteraceae Packera crocata
calculating outlier for group Asteraceae Erigeron pulcherrimus
calculating outlier for group Asteraceae Pterocaulon pycnostachyum
calculating outlier for group Asteraceae Garberia heterophylla
calculating outlier for group Asteraceae Eurybia divaricata
calculating outlier for group Asteraceae Chrysopsis subulata
calculating outlier for group Aster

/scratch/1616336.1.cds/ipykernel_1569578/2590339009.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_dict[name].loc[:, "eucl_distance"] = distances
/scratch/1616336.1.cds/ipykernel_1569578/2590339009.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_dict[name].loc[:, "outlier"] = outliers
/scratch/1616336.1.cds/ipykernel_1569578/2590339009.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value in

In [10]:
# recombine results to one dataframe 
df_combined = pd.concat(result_dict.values(), ignore_index=False)

In [76]:
df_combined.to_csv("pca_2d_outliers.csv")

In [22]:
df_combined.head()

,Unnamed: 0,0,1,label,eucl_distance,outlier
0,/projectnb/herbdl/data/kaggle-herbaria/herbari...,9.106315,0.102141,Asteraceae Ageratina jucunda,2.892152,False
164,/projectnb/herbdl/data/kaggle-herbaria/herbari...,14.531756,1.737664,Asteraceae Ageratina jucunda,2.912305,False
203,/projectnb/herbdl/data/kaggle-herbaria/herbari...,9.630124,3.463129,Asteraceae Ageratina jucunda,2.787500,False
205,/projectnb/herbdl/data/kaggle-herbaria/herbari...,9.757186,5.011175,Asteraceae Ageratina jucunda,3.961636,False
248,/projectnb/herbdl/data/kaggle-herbaria/herbari...,12.025526,7.481657,Asteraceae Ageratina jucunda,5.975991,True
266,/projectnb/herbdl/data/kaggle-herbaria/herbari...,13.476758,-0.772726,Asteraceae Ageratina jucunda,2.944615,False
320,/projectnb/herbdl/data/kaggle-herbaria/herbari...,13.369989,-1.148309,Asteraceae Ageratina jucunda,3.185871,False
404,/projectnb/herbdl/data/kaggle-herbaria/herbari...,13.432225,-0.663595,Asteraceae Ageratina jucunda,2.831921,False
417,/projectnb/herbdl/data/kaggle-herbaria/herbari...,13.725654,-3.259612,Asteraceae Ageratina jucunda,5.218803,False
495,/projectnb/herbdl/data/kaggle-herbaria/herbari...,11.765627,2.068515,Asteraceae Ageratina jucunda,0.566640,False


In [17]:
df_tsne = pd.read_csv("../asteraceae_tsne_astera_50_checkpoint-1300.csv")

In [20]:
df_tsne["pca_outlier"] = df_combined["outlier"]

In [24]:
df_tsne.head()

,Unnamed: 0,0,1,label,pca_outlier
0,/projectnb/herbdl/data/kaggle-herbaria/herbari...,26.651928,13.341015,Asteraceae Ageratina jucunda,False
1,/projectnb/herbdl/data/kaggle-herbaria/herbari...,18.600730,-18.657986,Asteraceae Carphephorus paniculatus,False
2,/projectnb/herbdl/data/kaggle-herbaria/herbari...,-29.582312,-21.055666,Asteraceae Crepis acuminata,False
3,/projectnb/herbdl/data/kaggle-herbaria/herbari...,9.445475,26.835894,Asteraceae Bidens vulgata,False
4,/projectnb/herbdl/data/kaggle-herbaria/herbari...,3.919897,8.282666,Asteraceae Arnica sororia,False


In [25]:
df_tsne.to_csv("../tsne_with_2d_pca_outlier.csv")

# Outliers PCA 

In [2]:
df_pca = pd.read_csv("../asteraceae_pca_10_astera_50_checkpoint-1300.csv")
df_pca.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,9,label
0,/projectnb/herbdl/data/kaggle-herbaria/herbari...,9.105993,0.101899,4.631554,-11.614634,-1.791820,-7.718238,2.456629,4.603161,1.909699,-7.485959,Asteraceae Ageratina jucunda
1,/projectnb/herbdl/data/kaggle-herbaria/herbari...,10.195303,-11.720765,5.667974,-0.047585,-0.236367,3.540408,-15.955474,-2.131101,3.381002,-2.250144,Asteraceae Carphephorus paniculatus
2,/projectnb/herbdl/data/kaggle-herbaria/herbari...,-5.752440,-7.106267,-7.364646,2.605586,-4.203525,14.398073,1.857420,0.202585,-0.226751,-1.229941,Asteraceae Crepis acuminata
3,/projectnb/herbdl/data/kaggle-herbaria/herbari...,3.561296,14.624349,-7.597177,-10.574511,4.946481,9.618900,-0.341040,-8.264402,-1.048600,5.325108,Asteraceae Bidens vulgata
4,/projectnb/herbdl/data/kaggle-herbaria/herbari...,-12.460394,-4.621159,4.782201,-7.747055,-8.316726,10.124874,2.436418,-10.581471,-6.922583,5.466442,Asteraceae Arnica sororia


In [3]:
# get list of labels - species 
pca_species_names = df_pca["label"].unique()

In [4]:
# for each label filter the df for the points 
pca_result_dict = {}
for name in pca_species_names:
    pca_result_dict[name] = df_pca[df_pca["label"] == name]

In [64]:
# calculate outliers for each species 
pca_centroids_dict = {}
for name in pca_species_names:
    print(f"calculating outlier for group {name}")
    temp = pca_result_dict[name].iloc[:, 1:11] # get only the components 
    X_pca = temp.to_numpy(dtype=float) # shape (n_samples, n_components)
    
    # calculate center of cluster and covariance matrix
    centroid = np.mean(X_pca, axis=0) 
    cov_matrix = np.cov(X_pca, rowvar=False)

    # calculate the inverse covariance matrix
    inv_cov_matrix = np.linalg.inv(cov_matrix)

    # redo this part
    group_distances = []
    for pt in X_pca:
        #eucl_distance = distance.euclidean(pt, centroid)
        maha_distance = distance.mahalanobis(pt, centroid, inv_cov_matrix)
        group_distances.append(maha_distance)

    # calculate threshold 
    #threshold = np.sqrt(chi2.ppf((1-0.05), df=10))    #degrees of freedom = number of variables
    threshold = np.percentile(group_distances, 95)
    print(threshold)
    
    # save the centroids
    pca_centroids_dict[name] = centroid
    
    # get the outliers 
    outliers = [True if d > threshold else False for d in group_distances]
    
    # save the results to the dataframe 
    pca_result_dict[name].loc[:, "maha_distance"] = group_distances 
    pca_result_dict[name].loc[:, "outlier"] = outliers

calculating outlier for group Asteraceae Ageratina jucunda
3.40573290473679
calculating outlier for group Asteraceae Carphephorus paniculatus
3.5319203578020986
calculating outlier for group Asteraceae Crepis acuminata
3.4925690727160825
calculating outlier for group Asteraceae Bidens vulgata
3.5436149071038443
calculating outlier for group Asteraceae Arnica sororia
3.5613637168916803
calculating outlier for group Asteraceae Brickellia longifolia
4.079369032268253
calculating outlier for group Asteraceae Liatris pilosa
3.801058490327732
calculating outlier for group Asteraceae Bidens beckii
4.121703190452968
calculating outlier for group Asteraceae Heterotheca pumila
3.598246621092178
calculating outlier for group Asteraceae Erigeron melanocephalus
3.467176148464633
calculating outlier for group Asteraceae Packera crocata
4.080754848070982
calculating outlier for group Asteraceae Erigeron pulcherrimus
3.7691449755956663
calculating outlier for group Asteraceae Pterocaulon pycnostachyum

In [65]:
df_combined_pca = pd.concat(pca_result_dict.values(), ignore_index=False)

In [66]:
df_combined_pca[df_combined_pca["outlier"] == True]

,Unnamed: 0,0,1,2,3,4,5,6,7,8,9,label,maha_distance,outlier
205,/projectnb/herbdl/data/kaggle-herbaria/herbari...,9.756797,5.010805,-3.678381,-7.961831,6.554263,-5.719462,-2.087889,-1.835466,4.922622,-9.003279,Asteraceae Ageratina jucunda,3.407011,True
686,/projectnb/herbdl/data/kaggle-herbaria/herbari...,1.001134,-7.611789,0.975769,-4.043293,-1.154724,0.134378,-7.427183,-8.101336,-0.884729,-5.162487,Asteraceae Carphephorus paniculatus,3.533118,True
198,/projectnb/herbdl/data/kaggle-herbaria/herbari...,0.874653,-9.304742,-5.470762,-5.597372,-3.018132,9.928748,-0.347517,2.504303,4.681208,1.281432,Asteraceae Crepis acuminata,3.493154,True
3,/projectnb/herbdl/data/kaggle-herbaria/herbari...,3.561296,14.624349,-7.597177,-10.574511,4.946481,9.618900,-0.341040,-8.264402,-1.048600,5.325108,Asteraceae Bidens vulgata,3.546270,True
4,/projectnb/herbdl/data/kaggle-herbaria/herbari...,-12.460394,-4.621159,4.782201,-7.747055,-8.316726,10.124874,2.436418,-10.581471,-6.922583,5.466442,Asteraceae Arnica sororia,3.563970,True
65,/projectnb/herbdl/data/kaggle-herbaria/herbari...,9.414876,1.309015,-3.166500,4.460389,3.549675,-0.440184,1.748916,-7.745335,9.212378,1.306718,Asteraceae Brickellia longifolia,4.082780,True
510,/projectnb/herbdl/data/kaggle-herbaria/herbari...,7.662725,-12.363817,-4.958069,7.990547,8.560899,-5.794503,-6.770853,-3.498354,1.256457,5.643928,Asteraceae Liatris pilosa,3.813124,True
93,/projectnb/herbdl/data/kaggle-herbaria/herbari...,-2.303639,8.382032,-3.401642,0.064308,11.868279,-3.382922,-2.617119,2.755310,-3.064926,-3.076519,Asteraceae Bidens beckii,4.136184,True
765,/projectnb/herbdl/data/kaggle-herbaria/herbari...,-8.543416,8.984144,4.470803,6.400980,-4.428992,-3.765203,-0.062852,-5.446758,7.272494,8.021980,Asteraceae Heterotheca pumila,3.603888,True
135,/projectnb/herbdl/data/kaggle-herbaria/herbari...,-13.292565,0.658025,15.266233,-1.193971,4.298266,8.932831,-8.343693,5.986741,-1.704357,-3.448751,Asteraceae Erigeron melanocephalus,3.467475,True


In [67]:
df_combined_pca.to_csv("pca_10_outliers_95th.csv")